# ECB:Integrity issues

Create a Python program that encrypts the messages in “message1.txt” and “message2.txt” using AES-128-ECB, using a key of your choosing.

The aforementioned text files portray messages exchanged between two parties for the setup of a meeting.

By concatenating a part of the ciphertext created from the first message and a part of the ciphertext created from the second message, create a third ciphertext which when decrypted shows that a meeting will take place at a date and time combination not shown in the first two messages. 
Print the resulting plaintext as well as the ciphertexts for message1.txt, message2.txt and the created message in the console, as a hexadecimal byte representation of the data.

Padding should not be considered for this exercise

In [1]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.padding import PKCS7
import secrets


In [2]:
def encrypt(plain: bytes, key: bytes):
    # Apply PKCS7 padding
    padder = PKCS7(128).padder()
    padded_data = padder.update(plain) + padder.finalize()
    
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    encryptor = cipher.encryptor()
    return encryptor.update(padded_data) + encryptor.finalize()

def decrypt(ciphertext: bytes, key: bytes):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    decryptor = cipher.decryptor()
    padded_plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    
    # Remove PKCS7 padding
    unpadder = PKCS7(128).unpadder()
    return unpadder.update(padded_plaintext) + unpadder.finalize()

def mix_ciphers(ct1: bytes, ct2: bytes):
    return ct1[:16*6] + ct2[16*6:]

In [3]:
def main(m1,m2):
    m1_plain = m1.encode()

    m2_plain = m2.encode()

    key = secrets.token_bytes(16)

    m1_cipher = encrypt(m1_plain, key)
    m2_cipher = encrypt(m2_plain, key)

    mixed = mix_ciphers(m1_cipher, m2_cipher)

    print(f"Key: {key.hex()}")

    print(f"Ciphertext 1: {m1_cipher.hex()}")
    print(f"Ciphertext 2: {m2_cipher.hex()}")
    print(f"Created ciphertext: {mixed.hex()}")
    print(f"Created message:\n{decrypt(mixed, key).decode()}", end='')

if __name__ == "__main__":
    message1 = "Meet me today at the church at 1130."
    message2 = "Meet me today at the market at 1530."
    main(message1,message2)



Key: 084c328125d477fd6d82bac64b83e032
Ciphertext 1: e65816381d8798963af967f87d5e2c2663b73c57d9bf49b60911607dc825bf7364c9600d905187fab91743b597e2f0bc
Ciphertext 2: e65816381d8798963af967f87d5e2c2650727f74db862ca90cce406efeae7ededf88fae8cb67b857bf560f74eb7eadf6
Created ciphertext: e65816381d8798963af967f87d5e2c2663b73c57d9bf49b60911607dc825bf7364c9600d905187fab91743b597e2f0bc
Created message:
Meet me today at the church at 1130.